In [1]:
from pycaret.classification import *
import pandas as pd
import os
import wandb

os.chdir('/home/antoine/gene_pheno_pred')
os.environ["CUDA_VISIBLE_DEVICES"]="2"
os.environ["WANDB_API_KEY"]="4e5748d6c6f3917c78cdc38a516a1bac776faf58"
wandb.login()
wandb.init(
    project="cigap-classif",
)


wandb: Currently logged in as: antoine-toffano (mlb). Use `wandb login --relogin` to force relogin


In [2]:
# Load training set
df1 = pd.read_csv('/home/antoine/gene_pheno_pred/TransE_2023-05-04_17-19-26_kg_val.csv' , header=0)
df2 = pd.read_csv('/home/antoine/gene_pheno_pred/TransE_2023-05-04_17-19-26_kg_test.csv' , header=0)
df = pd.concat([df1, df2], axis=0)

df['link'] = df['relation'].apply(lambda x: 1 if x != 'no_link_known' else 0)
df

,0,1,2,3,4,5,6,7,8,9,...,94,95,96,97,98,99,head,relation,tail,link
0,-2.651760e-40,-3.429000e-41,2.651760e-40,2.651760e-40,-1.023535e-39,1.328263e-39,-1.096247e-39,3.041970e-40,-3.041970e-40,3.429000e-41,...,5.658560e-40,-8.804160e-40,-5.658560e-40,-8.804160e-40,8.804160e-40,-8.804160e-40,https://wormbase.org/wbdata/pheno-352464,no_link_known,https://wormbase.org/wbdata/pheno-249726,0
1,-1.852055e-02,-1.852055e-02,-1.852055e-02,1.852055e-02,-1.852055e-02,-1.852055e-02,-1.852055e-02,1.852055e-02,-1.852055e-02,1.852055e-02,...,7.091515e-03,7.091515e-03,-1.054280e-02,1.054280e-02,-1.054280e-02,1.054280e-02,https://wormbase.org/wbdata/pheno-368707,http://semanticscience.org/resource/SIO_000281,https://wormbase.org/species/all/phenotype/WBP...,1
2,6.551130e-03,-4.301126e-16,-1.286188e-02,4.301126e-16,6.551130e-03,3.461342e-05,1.641598e-02,4.301126e-16,-4.301126e-16,9.878389e-06,...,9.292338e-03,1.606221e-02,-1.606221e-02,9.292338e-03,-1.606221e-02,9.292338e-03,https://wormbase.org/wbdata/pheno-337158,http://semanticscience.org/resource/SIO_000281,https://wormbase.org/species/all/phenotype/WBP...,1
3,1.285421e-02,-1.111513e-01,3.242642e-01,2.040135e-01,-3.114431e-01,1.425715e-02,1.980742e-01,9.400842e-02,-2.113062e-01,4.629730e-02,...,-7.922775e-03,1.460967e-02,7.922775e-03,-7.922775e-03,-1.460967e-02,1.460967e-02,https://wormbase.org/wbdata/dev-50559,http://www.semanticweb.org/needed-terms#004,https://wormbase.org/species/all/expr_pattern/...,1
4,-3.533729e-02,-2.895543e-02,4.201949e-01,2.895544e-02,-2.988258e-02,-2.897010e-02,4.705756e-02,2.954788e-02,-2.463294e-02,-4.839125e-01,...,5.886103e-03,5.886103e-03,-1.193518e-02,5.886103e-03,-5.886103e-03,5.886103e-03,https://wormbase.org/wbdata/pheno-111712,http://www.semanticweb.org/needed-terms#001,https://wormbase.org/species/c_elegans/gene/WB...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234297,8.892616e-03,8.892616e-03,-8.892616e-03,-8.892176e-03,3.804456e-01,8.724516e-03,7.835008e-02,-8.892335e-03,6.774163e-01,-8.892616e-03,...,-2.389259e-02,-9.936046e-01,1.080367e-02,-1.080367e-02,1.044651e-02,-1.136564e-02,https://wormbase.org/wbdata/dev-16653,no_link_known,https://wormbase.org/wbdata/pheno-154583,0
234298,-1.036218e-02,-1.036218e-02,-1.036218e-02,1.036218e-02,-1.036218e-02,-1.036218e-02,-1.036218e-02,1.036218e-02,-1.036218e-02,1.036218e-02,...,8.737751e-03,8.737751e-03,-3.985811e-03,8.737751e-03,-3.985811e-03,4.050367e-03,https://wormbase.org/wbdata/pheno-232021,http://www.semanticweb.org/needed-terms#001,https://wormbase.org/species/c_elegans/gene/WB...,1
234299,-2.370377e-02,-2.370377e-02,3.308815e-02,2.370377e-02,-3.308815e-02,-3.308815e-02,-2.370377e-02,3.308815e-02,-3.308815e-02,2.370377e-02,...,-3.643983e-27,-3.643983e-27,3.643983e-27,-3.643983e-27,3.643983e-27,-3.643983e-27,https://wormbase.org/wbdata/pheno-315170,no_link_known,https://wormbase.org/wbdata/pheno-240091,0
234300,-2.974385e-03,-2.974385e-03,-1.042132e-02,2.974385e-03,-2.974385e-03,-2.974385e-03,-3.247903e-02,1.042132e-02,-2.974385e-03,2.961988e-03,...,1.626528e-01,1.036491e-01,-1.207425e-01,-1.405245e-01,1.510951e-01,1.397504e-01,https://wormbase.org/wbdata/pheno-187895,no_link_known,https://wormbase.org/wbdata/dev-45234,0


In [3]:
df2 = df

In [4]:
df = df.drop('relation', axis=1)

In [5]:
# Experiment setup
s = setup(df, target = 'link', fold_strategy = 'stratifiedkfold', fold=10, train_size = 0.8, n_jobs=-1, system_log=True, use_gpu = True, log_experiment='wandb')
exp = ClassificationExperiment()

[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recomp

MemoryError: Unable to allocate 15.2 GiB for an array with shape (100, 40781969) and data type float32

# K Neighbors Classifier

In [ ]:
knn = create_model('knn')

In [ ]:
plot_model(knn, plot = 'confusion_matrix')

In [ ]:
plot_model(knn, plot = 'error')

In [ ]:
plot_model(knn, plot = 'parameter')

In [ ]:
save_model(knn, 'models_classif/convkb/binary_emb_val/knn')

# Naive Bayes

In [ ]:
nb = create_model('nb')

In [ ]:
plot_model(nb, plot = 'confusion_matrix')

In [ ]:
plot_model(nb, plot = 'error')

In [ ]:
plot_model(nb, plot = 'parameter')

In [ ]:
save_model(nb, 'models_classif/convkb/binary_emb_val/nb')

# Decision Tree Classifier

In [ ]:
dt = create_model('dt')

In [ ]:
plot_model(dt, plot = 'confusion_matrix')

In [ ]:
plot_model(dt, plot = 'error')

In [ ]:
plot_model(dt, plot = 'parameter')

In [ ]:
save_model(dt, 'models_classif/convkb/binary_emb_val/dt')

# SVM - Linear Kernel

In [ ]:
svm = create_model('svm')

In [ ]:
plot_model(svm, plot = 'confusion_matrix')

In [ ]:
plot_model(svm, plot = 'error')

In [ ]:
plot_model(svm, plot = 'parameter')

In [ ]:
save_model(svm, 'models_classif/convkb/binary_emb_val/svm')

# Ridge Classifier

In [ ]:
ridge = create_model('ridge')

In [ ]:
plot_model(ridge, plot = 'confusion_matrix')

In [ ]:
plot_model(ridge, plot = 'error')

In [ ]:
plot_model(ridge, plot = 'parameter')

In [ ]:
save_model(ridge, 'models_classif/convkb/binary_emb_val/ridge')

# Random Forest

In [ ]:
rf = create_model('rf')

In [ ]:
plot_model(rf, plot = 'confusion_matrix')

In [ ]:
plot_model(rf, plot = 'error')

In [ ]:
plot_model(rf, plot = 'parameter')

In [ ]:
save_model(rf, 'models_classif/convkb/binary_emb_val/rf')

# Quadratic Discriminant Analysis

In [ ]:
qda = create_model('qda')

In [ ]:
plot_model(qda, plot = 'confusion_matrix')

In [ ]:
plot_model(qda, plot = 'error')

In [ ]:
plot_model(qda, plot = 'parameter')

In [ ]:
save_model(qda, 'models_classif/convkb/binary_emb_val/qda')

# Linear Discriminant Analysis

In [ ]:
lda = create_model('lda')

In [ ]:
plot_model(lda, plot = 'confusion_matrix')

In [ ]:
plot_model(lda, plot = 'error')

In [ ]:
plot_model(lda, plot = 'parameter')

In [ ]:
save_model(lda, 'models_classif/convkb/binary_emb_val/lda')

# Extra Trees

In [ ]:
et = create_model('et')

In [ ]:
plot_model(et, plot = 'confusion_matrix')

In [ ]:
plot_model(et, plot = 'error')

In [ ]:
plot_model(et, plot = 'parameter')

In [ ]:
save_model(et, 'models_classif/convkb/binary_emb_val/et')

# Light Gradient Boosting Machine

In [ ]:
lightgbm = create_model('lightgbm')

In [ ]:
plot_model(lightgbm, plot = 'confusion_matrix')

In [ ]:
plot_model(lightgbm, plot = 'error')

In [ ]:
plot_model(lightgbm, plot = 'parameter')

In [ ]:
save_model(lightgbm, 'models_classif/convkb/binary_emb_val/lightgbm')

# MLP Classifier

In [ ]:
mlp = create_model('mlp')

In [ ]:
plot_model(mlp, plot = 'confusion_matrix')

In [ ]:
plot_model(mlp, plot = 'error')

In [ ]:
plot_model(mlp, plot = 'parameter')

In [ ]:
save_model(mlp, 'models_classif/convkb/binary_emb_val/mlp')